# Team Project 
## Member
- Hyeonseo Park (19102090)
- Younghwan Pahn (19102091)
### Subject 
Digital signature proxy in a cloud environment

In [337]:
%/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
%pip install pi-heaan numpy pandas pickle
%%python3 -m pip install cryptography



UsageError: Line magic function `%/Library/Developer/CommandLineTools/usr/bin/python3` not found.


In [338]:
import piheaan as heaan
import json
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import numpy as np
import pandas as pd
import pickle
from os import urandom
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.asymmetric import padding as asymmetric_padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


## User Class for simulating 
### Purpose
- Key Management
- Encrypt Secure Information
- Decrypt Secure Information

In [339]:
class User:
    def __init__(self, _id):
        params = heaan.ParameterPreset.FGa
        self.context = heaan.make_context(params)
        heaan.make_bootstrappable(self.context)
        self.id = _id
        key_file_path = _id
        heaan_private_key = heaan.SecretKey(self.context) # create secret key
        os.makedirs(key_file_path, mode=0o775, exist_ok=True)
        
        heaan_private_key.save(key_file_path+"/secretkey.bin") # save secret key
        key_generator = heaan.KeyGenerator(self.context, heaan_private_key) # create public key
        key_generator.gen_common_keys()
        key_generator.save(key_file_path+"/") # save public key   

        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        self.public_key = self.private_key.public_key()     
    
    def readHeaanKeyPair(self, _id):
        key_file_path = "./" + _id
        sk = heaan.SecretKey(self.context,key_file_path+"/secretkey.bin") # load secret key
        pk = heaan.KeyPack(self.context, key_file_path+"/") # load public key
        pk.load_enc_key()
        pk.load_mult_key()
        enc = heaan.Encryptor(self.context) # for encrypt
        dec = heaan.Decryptor(self.context) # for decrypt
        eval = heaan.HomEvaluator(self.context,pk)
        return sk, pk, enc, dec, self.context, eval
    
    def encryptData(self, plainTexts):
        cipherTexts = []
        for data in plainTexts:
            cipherText = self.public_key.encrypt(
                data.encode(),  
                asymmetric_padding.OAEP(
                    mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            cipherTexts.append(cipherText)
        return cipherTexts
    
    def decryptData(self, cipherTexts):
        plainTexts = []
        for edata in cipherTexts:
            plainText = self.private_key.decrypt(
                edata,
                asymmetric_padding.OAEP(
                    mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            plainTexts.append(plainText.decode())
        return plainTexts
    
    def decryptData_SymmetricKey(self, key, cipherText):
        iv = cipherText[:16]  # IV는 암호문의 처음 16바이트에 있습니다.
        actual_ciphertext = cipherText[16:]
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        decrypted_padded = decryptor.update(actual_ciphertext) + decryptor.finalize()
        # 패딩을 제거합니다.
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        decrypted = unpadder.update(decrypted_padded) + unpadder.finalize()
        return decrypted.decode()

class Holder(User):
    def __init__(self, _id, email, name):
        super().__init__(_id)
        self.holder_info = [_id, email, name]
        self.symmetricKey = urandom(32)
    
    def encryptData_SymmetricKey(self, key, plainText):
        iv = urandom(16)  
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        
        padder = padding.PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(plainText.encode()) + padder.finalize()
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
        return iv + ciphertext  
    
    def encryptData_OtherKey(self, key, plainText, isBytes):
        data = plainText if isBytes else plainText.encode()
        cipherText = key.encrypt(
            data,  
            asymmetric_padding.OAEP(
                mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )   
        return cipherText
    

class CA(User):
    def __init__(self, _id):
        super().__init__(_id)
    
    def decryptData(self, cipherText):
        plainText = self.private_key.decrypt(
            cipherText,
            asymmetric_padding.OAEP(
                mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return plainText
    def encryptData(self, plainText, isBytes):
        data = plainText if isBytes else plainText.encode()
        cipherText = self.public_key.encrypt(
            data,  
            asymmetric_padding.OAEP(
                mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        return cipherText



## Send Digital Envelop To CA
- Assuming the reliability of the information and identity of the holder, the certification issuance process proceeds. ( Issuer is not included in our project )

- Holder encrypt Holder's Client ID with Holder's public key 
- Holder encrypt Public Key with Holder's Symmetric Key
- Symmetric key is encrypted by CA’s Public Key.  
- Holder converts his confidential information to int after encryption so that it can be used for homomorphic encryption.
- However, because this integer is very large, it cannot be used as a float, so the integer is divided into 15 digits and made into chunks.
- Send Digital Envelope to CA.

In [340]:
# Because the encrypted byte string converted to an integer is too long, dividing it is necessary.
def split_large_integer(n, chunk_size=15):
    n_str = str(n)  
    chunks = [] 
    for i in range(len(n_str), 0, -chunk_size):
        start = max(0, i - chunk_size)  
        chunks.append(int(n_str[start:i])) 

    return chunks[::-1]

def serialize_public_key(public_key):
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return pem.decode('utf-8') 

In [341]:
holder = Holder("990816-1","lopahn2@gmail.com", "반영환")
_CA = CA("Government")


# client already know about CA's public Key
PUca = _CA.public_key

# Holder encrypt his Client ID
cipherTextOfHolderInfo = holder.encryptData(holder.holder_info)
parseInt_CipherTextOfHolderInfo = [int.from_bytes(b, 'big') for b in cipherTextOfHolderInfo]

chunks = {
    0:[],
    1:[],
    2:[]
}

for i in range(len(parseInt_CipherTextOfHolderInfo)):
    chunks[i] = split_large_integer(parseInt_CipherTextOfHolderInfo[i])

# For HE Calculation, Making client data slot
log_slots = 15
num_slots = 2**log_slots

messages = {
    0:heaan.Message(log_slots),
    1:heaan.Message(log_slots),
    2:heaan.Message(log_slots)
}

for k, v in chunks.items():
    for i in range(len(v)):
        messages[k][i] = complex(v[i],0)


PUho = serialize_public_key(holder.public_key)
puhoJson = json.dumps(PUho)

encryptedpuhoJson = holder.encryptData_SymmetricKey(holder.symmetricKey, puhoJson)
encryptedSymmetricKey = holder.encryptData_OtherKey(PUca, holder.symmetricKey, True)

# Holder send digital envelop to CA
envelop = {
    "message": messages,
    "public_key" : encryptedpuhoJson,
    "symmetric_key" : encryptedSymmetricKey
}



## Receive Digital Envelop From Holder
- CA Decrypt E(SY_KEY) by CA’s Private Key

-  CA Decrypt E(Message) By SY_KEY
-  CA obtain Holder’s public key and Encrypted Client ID
-  This stage, CA couldn’t know about Holder’s private
information.
- CA encrypt Holder's SY_KEY with CA's public Key and parse to integer.
- CA makes the large integer 15 digits long to match the number of digits in the user's encrypted message.
- CA add ciphertext and encrypted symmetric key drived addend. Then make response message for response to the Holder

In [342]:
plainMessages = envelop["message"]
symmetricKey = _CA.decryptData(envelop["symmetric_key"])
receivedPublicKey = _CA.decryptData_SymmetricKey(symmetricKey, envelop["public_key"])

sk, pk, enc, dec, context, eval = _CA.readHeaanKeyPair(_CA.id)
cipherMessages = {
  0: heaan.Ciphertext(context),
  1: heaan.Ciphertext(context),
  2: heaan.Ciphertext(context)
}

for i in range(len(plainMessages)):
  enc.encrypt(plainMessages[i], pk, cipherMessages[i])

symmetricKeyDrivedAddend = heaan.Message(log_slots)

cutoff = 602
cnt = 0
while cnt != len(symmetricKeyDrivedAddend):
  symmetricKeyDrivedAddend[cnt] = int.from_bytes(_CA.encryptData(str(symmetricKey[(cnt % 32)]).encode(), True),"little") // 10 ** cutoff
  cnt += 1

responseMessage = {
  0: heaan.Ciphertext(context),
  1: heaan.Ciphertext(context),
  2: heaan.Ciphertext(context),
}

for i in range(len(cipherMessages)):
  eval.add(cipherMessages[i], symmetricKeyDrivedAddend, responseMessage[i])

## Verify User Information
- Holder send their plain secure information (id, email, name)

- Holder send received message from CA to verifier
- Holder encrypt symmetric key which was sent to CA with holder's private key
- Verifier search Holder's public key with id from plain secure information
- Verifier encrypt Holder's secure information as samely manner that holder did.
- Verifier send holder's received message and holder's derypted symmetric key to CA
- CA responds to the message by doing the opposite of how it issued the certificate. 
- Verifier check CA's response and encrypted Holder's information are same or not.
- If same, Holder is verified user. Or not, Holder's verification is failed.
